## C Histogram Program

In [1]:
%%writefile C_Histogram.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#define BINS 10
#define VEC_SIZE 268435456

int main()
{
  clock_t start = clock();
  int *vector=(int *)malloc(sizeof(int) * VEC_SIZE);
  for (int i = 0; i <VEC_SIZE; i++)
  {
    vector[i] = i;
  }
  int histogram[BINS] = {0};

  for (int i = 0; i < VEC_SIZE;i++)
  {
    int bin = vector[i] % BINS;
    histogram[bin]++;
  }
  clock_t end = clock();
  double time =((double)(end-start)) / CLOCKS_PER_SEC;
  printf("Historgram Counter:\n");
  for(int i=0; i < BINS; i++)
  {
    printf("Bin %d: %d\n", i,histogram[i] );
  }
  printf("execution time:%f seconds\n",time);
  size_t err_count = 0;
  int lbin=VEC_SIZE/10;
  int hbin=lbin+1;
  int thresh=VEC_SIZE%10;
  for (size_t i=0; i<BINS; i++){
    if (i<thresh){
      if (hbin!=histogram[i])
        err_count++;
    }
    else{
      if (lbin!=histogram[i])
        err_count++;
    }
  }

  printf("no. of errors:%ld\n",err_count);
  free(vector);
  return 0;
}

Writing C_Histogram.c


In [2]:
%%shell
gcc C_Histogram.c -o C_Histogram
./C_Histogram

Historgram Counter:
Bin 0: 26843546
Bin 1: 26843546
Bin 2: 26843546
Bin 3: 26843546
Bin 4: 26843546
Bin 5: 26843546
Bin 6: 26843545
Bin 7: 26843545
Bin 8: 26843545
Bin 9: 26843545
execution time:2.772377 seconds
no. of errors:0


In [ ]:
%%shell
count=30
for i in $(seq $count); do
    ./C_Histogram
done

execution time:2.214624 seconds
execution time:2.238716 seconds
execution time:2.200602 seconds
execution time:2.694822 seconds
execution time:2.212267 seconds
execution time:2.196411 seconds
execution time:2.203299 seconds
execution time:2.193172 seconds
execution time:2.656205 seconds
execution time:2.252391 seconds
execution time:2.202794 seconds
execution time:2.213818 seconds
execution time:2.203160 seconds
execution time:2.493078 seconds
execution time:2.292633 seconds
execution time:2.208325 seconds
execution time:2.196955 seconds
execution time:2.206388 seconds
execution time:2.375831 seconds
execution time:2.429471 seconds
execution time:2.197898 seconds
execution time:2.205885 seconds
execution time:2.210731 seconds
execution time:2.217466 seconds
execution time:2.625715 seconds
execution time:2.199572 seconds
execution time:2.209313 seconds
execution time:2.213637 seconds
execution time:2.197803 seconds
execution time:2.692892 seconds


## CUDA Histogram Program

In [3]:
%%writefile C_CudaHistogram.cu
#include <stdio.h>
#include <cuda.h>
#include <time.h>
#define VEC_SIZE 268435456
#define BINS 10
#define threadsPerBlock 256

__global__ void histogramFunc(int *vector,int *histogram, int n)
{
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  if(idx < n)
  {
    int bin = vector[idx] % BINS;
    atomicAdd(&histogram[bin],1);
  }
}

int main()
{
  clock_t start = clock();
  int *vector,*histogram;
  int device = 0;
  cudaSetDevice(device);
  cudaMallocManaged(&vector,VEC_SIZE * sizeof(int));
  cudaMallocManaged(&histogram,VEC_SIZE * sizeof(int));
  cudaMemAdvise(vector,VEC_SIZE*sizeof(int),cudaMemAdviseSetPreferredLocation,device);
  cudaMemAdvise(histogram,BINS*sizeof(int),cudaMemAdviseSetPreferredLocation,device);
  cudaMemAdvise(histogram,BINS*sizeof(int),cudaMemAdviseSetAccessedBy,device);

  for(int i=0; i<VEC_SIZE;i++)
  {
    vector[i]=i;
  }
  int blocks = (VEC_SIZE + threadsPerBlock-1) / threadsPerBlock;
  histogramFunc<<<blocks,threadsPerBlock>>>(vector,histogram,VEC_SIZE);
  cudaDeviceSynchronize();
  clock_t end = clock();

  for(int i = 0 ; i < BINS;i++)
  {
    printf("Bin %d: %d\n",i,histogram[i]);
  }
  double time =((double)(end-start)) / CLOCKS_PER_SEC;
  printf("execution time:%f seconds\n",time);
  size_t err_count = 0;
  int lbin=VEC_SIZE/10;
  int hbin=lbin+1;
  int thresh=VEC_SIZE%10;
  for (size_t i=0; i<BINS; i++){
    if (i<thresh){
      if (hbin!=histogram[i])
        err_count++;
    }
    else{
      if (lbin!=histogram[i])
        err_count++;
    }
  }

  printf("no. of errors:%ld\n",err_count);


  cudaFree(vector);
  cudaFree(histogram);

  return 0;

}

Writing C_CudaHistogram.cu


In [4]:
!nvcc -arch=sm_75 C_CudaHistogram.cu -o C_CudaHistogram
!./C_CudaHistogram

Bin 0: 26843546
Bin 1: 26843546
Bin 2: 26843546
Bin 3: 26843546
Bin 4: 26843546
Bin 5: 26843546
Bin 6: 26843545
Bin 7: 26843545
Bin 8: 26843545
Bin 9: 26843545
execution time:1.723160 seconds
no. of errors:0


In [ ]:
%%shell
count=30
for i in $(seq $count); do
    ./C_CudaHistogram
done

execution time:1.708912 seconds
execution time:1.643663 seconds
execution time:1.747311 seconds
execution time:1.818090 seconds
execution time:1.644151 seconds
execution time:1.643448 seconds
execution time:1.640953 seconds
execution time:1.609790 seconds
execution time:1.628516 seconds
execution time:1.702617 seconds
execution time:1.776193 seconds
execution time:1.626385 seconds
execution time:1.634932 seconds
execution time:1.650545 seconds
execution time:1.657738 seconds
execution time:1.662367 seconds
execution time:1.770837 seconds
execution time:1.714365 seconds
execution time:1.642696 seconds
execution time:1.654965 seconds
execution time:1.648395 seconds
execution time:1.670149 seconds
execution time:1.676327 seconds
execution time:1.752822 seconds
execution time:1.792196 seconds
execution time:1.662551 seconds
execution time:1.674336 seconds
execution time:1.661690 seconds
execution time:1.712766 seconds
execution time:1.638753 seconds
